In [20]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
from tensorflow.keras import Model, layers
import numpy as np

In [26]:
# MNIST Dataset
num_classes = 10
num_features = 784

learning_rate = 0.001
training_steps = 1000
batch_size = 32
display_step = 100

num_input = 28
timesteps = 28
num_units = 32

In [27]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
x_train, x_test = x_train.reshape([-1, 28, 28]), x_test.reshape([-1, num_features])
x_train, x_test = x_train / 255., x_test / 255.

In [28]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [39]:
class BiRNN(Model):
    def __init__(self):
        super(BiRNN, self).__init__()
        lstm_fw = layers.LSTM(units = num_units)
        lstm_bw = layers.LSTM(units = num_units, go_backwards = True)
        
        self.bi_lstm = layers.Bidirectional(lstm_fw, backward_layer = lstm_bw)
        self.out = layers.Dense(num_classes)
    
    def call(self, x, is_training = False):
        x = self.bi_lstm(x)
        x = self.out(x)
        if not is_training:
            x = tf.nn.softmax(x)
        return x

In [40]:
birnn_net = BiRNN()

In [41]:
def cross_entropy_loss(x, y):
    y = tf.cast(y, tf.int64)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = x)
    return tf.reduce_mean(loss)

def accuracy(y_pred, y_true):
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis = -1)

In [42]:
optimizer = tf.optimizers.Adam(learning_rate)

In [ ]:
def run_optimization(x, y):
    with tf.GradientTape() as g:
        pred = birnn_net(x, is_training = True)
        loss = cross_entropy_loss(pred, y)
        
    trainable_variables = birnn_net.trainable_variables
    gradients = g.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))